# Movie Reviews Sentiment Analysis

### Problem Statement
- In this project, we try to estimate the sentiment from a movie review.

### Dataset
- Source - https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
- The data contains 50,000 reviews, 25,000 of them are positive and 25,000 of them are negative.

In this notebook, we perform training on following models - 
- Logistic Regresssion

### Importing required packages

In [1]:
import numpy as np 
import scipy
import pandas as pd
import re
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from nltk.stem.porter import PorterStemmer
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline  
from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
import pickle
import os

### Importing data

In [2]:
df = pd.read_csv('data/imdb_data.csv')
df.head()

,Review,Rating,Sentiment
0,Imagine The Big Chill with a cast of twenty-so...,2,0
1,I'd have to say that I've seen worse Sci Fi Ch...,3,0
2,Director Fabio Barreto got a strange Academy N...,1,0
3,Pretty bad PRC cheapie which I rarely bother t...,4,0
4,This is a very intriguing short movie by David...,8,1


In [3]:
print(f'Shape = {df.shape}')

Shape = (50000, 3)


### Preprocessing text

In [4]:
def text_preprocessing(text):
    # Replacing n't with not since it could be really important in sentiment analysis
    text = re.sub("n't", ' not ', text)
    # Removing URLs
    text = re.sub('(http).*\/', ' ', text)
    # Removing HTML tags
    text = re.sub('<[^>]*>', '', text)
    # Extracting emoticons
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|P|D|]|})', text)
    # Removing punctuations
    text = re.sub('[\W]+', ' ', text.lower())
    # Adding emoticons at end and converting :-) to :)
    text = text + ' ' + ' '.join(emoticons).replace('-', '')
    return text   

In [5]:
df['Review'] = df['Review'].apply(text_preprocessing)
df.head()

,Review,Rating,Sentiment
0,imagine the big chill with a cast of twenty so...,2,0
1,i d have to say that i ve seen worse sci fi ch...,3,0
2,director fabio barreto got a strange academy n...,1,0
3,pretty bad prc cheapie which i rarely bother t...,4,0
4,this is a very intriguing short movie by david...,8,1


### Creating Feature Matrix and Target Vector

In [6]:
X = df['Review'].values 
y = df['Sentiment'].values 

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42, stratify=y)

Tokenizer-1 : we simply split the text 

In [8]:
def tokenizer(text):
    return text.split()

Tokenizer-2 : we use a tokenizer which contains stems of words as tokens. We use PorterStemmer() module to stem the words.

In [9]:
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

Removing Stopwords

In [10]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mndpp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
print(f'Number of stopwords = {len(stop)}')

Number of stopwords = 179


In [12]:
print(f'Stopwords = {stop}')

Stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so

There are certain stopwords which are relevant so we don't to remove those words

In [13]:
relevant_stopwords = ['no', 'not', 'ain', 'don', "don't", 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
new_stop = list(set(stop) - set(relevant_stopwords))
print(f'Stopwords remaining = {len(new_stop)}')

Stopwords remaining = 140


## Models

In [14]:
accuracy_models = {}

Post Training

In [15]:
def post_training(rs, model_name):
    print(f'Best Accuracy = {rs.best_score_:.4f}')
    print(f'Best Parameters = \n{rs.best_params_}')
    model = rs.best_estimator_
    model.fit(X_train, y_train)
    print(f'Accuracy on test set = {model.score(X_test, y_test)}')
    accuracy_models[model_name] = model.score(X_test, y_test)
    if not os.path.exists('models'):
        os.mkdir('models')
    model_filename = f'models/{model_name}_model.pickle'
    pickle.dump(model, open(model_filename, 'wb'))
    return model

### 1. Logistic Regression

In [16]:
model_name = 'logistic_regression'
tfidf = TfidfVectorizer(lowercase = False,
                        preprocessor=None,
                        min_df=5)
lr_tfidf_pipe = Pipeline([
    ('vect', tfidf),
    ('clf', LogisticRegression(solver='liblinear'))
])
np.random.seed(42)
param_range = scipy.stats.loguniform(0.0001, 1000.0)
param_grid_rs = [
    {
        'vect__ngram_range' : [(1, 1), (1, 2)],
        'vect__stop_words'  : [None, new_stop],
        'vect__tokenizer'   : [tokenizer, tokenizer_porter],
        'clf__penalty'      : ['l1', 'l2'],
        'clf__C'            : param_range
    },
    {
        'vect__ngram_range' : [(1, 1), (1, 2)],
        'vect__stop_words'  : [None, new_stop],
        'vect__tokenizer'   : [tokenizer, tokenizer_porter],
        'vect__use_idf'     : [False],
        'vect__norm'        : [None],
        'clf__penalty'      : ['l1','l2'],
        'clf__C'            : param_range
    }
]
rs_lr = RandomizedSearchCV(lr_tfidf_pipe, param_grid_rs,
                           scoring='accuracy', cv=5,
                           verbose = 2, n_jobs= -1,
                           random_state = 42, n_iter = 50)
rs_lr.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


c:\Users\mndpp\Desktop\Github_Projects\01_movie_review_sentiment\venv\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('vect',
                                              TfidfVectorizer(lowercase=False,
                                                              min_df=5)),
                                             ('clf',
                                              LogisticRegression(solver='liblinear'))]),
                   n_iter=50, n_jobs=-1,
                   param_distributions=[{'clf__C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000027C933CC6D0>,
                                         'clf__penalty': ['l1', 'l2'],
                                         'vect__ngram_range': [(1, 1), (1, 2)],
                                         'vect...
                                                               'nor', 'through',
                                                               'm', 'my',
                                                               'same', "you're",
                                                               'myself',
                                                               'theirs', 'do',
                                                               'the', 'herself',
                                                               'doing', 'only',
                                                               'here', 'll',
                                                               'our', 'for',
                                                               "should've",
                                                               'most', 'each',
                                                               'i', 'at',
                                                               'their', 'then', ...]],
                                         'vect__tokenizer': [<function tokenizer at 0x0000027C92EC64D0>,
                                                             <function tokenizer_porter at 0x0000027C92EC6B90>],
                                         'vect__use_idf': [False]}],
                   random_state=42, scoring='accuracy', verbose=2)

In [17]:
lr_model = post_training(rs_lr, model_name)

Best Accuracy = 0.9030
Best Parameters = 
{'clf__C': 0.05809926352311592, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 2), 'vect__norm': None, 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x0000027C92EC64D0>, 'vect__use_idf': False}


c:\Users\mndpp\Desktop\Github_Projects\01_movie_review_sentiment\venv\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Accuracy on test set = 0.90188


### 2. Naive Bayes

In [18]:
model_name = 'naive_bayes'
tfidf = TfidfVectorizer(lowercase = False,
                        preprocessor=None,
                        min_df=5)
nb_tfidf_pipe = Pipeline([
    ('vect', tfidf),
    ('clf', MultinomialNB())
])
np.random.seed(42)
param_range = scipy.stats.loguniform(0.00001, 10000.0)
param_grid_rs = [
    {
        'vect__ngram_range' : [(1, 1), (1, 2)],
        'vect__stop_words'  : [None, new_stop],
        'vect__tokenizer'   : [tokenizer, tokenizer_porter],
        'clf__alpha'        : param_range
    },
    {
        'vect__ngram_range' : [(1, 1), (1, 2)],
        'vect__stop_words'  : [None, new_stop],
        'vect__tokenizer'   : [tokenizer, tokenizer_porter],
        'vect__use_idf'     : [False],
        'vect__norm'        : [None],
        'clf__alpha'        : param_range
    }
]
rs_nb = RandomizedSearchCV(nb_tfidf_pipe, param_grid_rs,
                           scoring='accuracy', cv=5,
                           verbose = 2, n_jobs= -1,
                           random_state = 42, n_iter = 50)
rs_nb.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


c:\Users\mndpp\Desktop\Github_Projects\01_movie_review_sentiment\venv\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('vect',
                                              TfidfVectorizer(lowercase=False,
                                                              min_df=5)),
                                             ('clf', MultinomialNB())]),
                   n_iter=50, n_jobs=-1,
                   param_distributions=[{'clf__alpha': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000027C9345CAC0>,
                                         'vect__ngram_range': [(1, 1), (1, 2)],
                                         'vect__stop_words': [None,
                                                              ['he', 'both',
                                                               'any', 'has',
                                                               '...
                                                               'nor', 'through',
                                                               'm', 'my',
                                                               'same', "you're",
                                                               'myself',
                                                               'theirs', 'do',
                                                               'the', 'herself',
                                                               'doing', 'only',
                                                               'here', 'll',
                                                               'our', 'for',
                                                               "should've",
                                                               'most', 'each',
                                                               'i', 'at',
                                                               'their', 'then', ...]],
                                         'vect__tokenizer': [<function tokenizer at 0x0000027C92EC64D0>,
                                                             <function tokenizer_porter at 0x0000027C92EC6B90>],
                                         'vect__use_idf': [False]}],
                   random_state=42, scoring='accuracy', verbose=2)

In [19]:
model_nb = post_training(rs_nb, 'naive_bayes')

Best Accuracy = 0.8878
Best Parameters = 
{'clf__alpha': 0.4869768770261902, 'vect__ngram_range': (1, 2), 'vect__stop_words': ['he', 'both', 'any', 'has', 'ma', 'itself', 'nor', 'through', 'm', 'my', 'same', "you're", 'myself', 'theirs', 'do', 'the', 'herself', 'doing', 'only', 'here', 'll', 'our', 'for', "should've", 'most', 'each', 'i', 'at', 'their', 'then', 'she', 'until', 'ourselves', 'if', "it's", 'this', 'd', 'when', 'before', 't', 'so', "that'll", 'or', 'who', 'am', 'against', 'does', 'again', 'been', 'on', 'were', 'should', 'there', 'from', 'some', "you'll", 'yours', 'once', 'but', 'now', 'those', 'than', 'off', 'being', 'these', 'and', 'an', 'you', 'which', 'few', 'will', 'further', 'how', 'more', 'in', 'himself', 'did', 'him', 'themselves', 'while', 'own', 'was', 've', "you'd", 'hers', 'what', 'having', 'them', 'had', 'such', 'up', 'about', 'its', 'during', 'over', 'where', 'o', 'ours', 'very', 'that', 'her', 'we', 'can', 'other', "she's", 'whom', 'just', 'is', 'a', 'his', '

c:\Users\mndpp\Desktop\Github_Projects\01_movie_review_sentiment\venv\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Accuracy on test set = 0.88232


### 3. SVM

In [20]:
model_name = 'SVM'
tfidf = TfidfVectorizer(lowercase = False,
                        preprocessor=None,
                        min_df=5)
svm_tfidf_pipe = Pipeline([
    ('vect', tfidf),
    ('clf', SVC())
])
np.random.seed(42)
param_range = scipy.stats.loguniform(0.00001, 10000.0)
param_distributions = [
    {
        'vect__ngram_range' : [(1, 1), (1, 2)],
        'vect__stop_words'  : [None, new_stop],
        'vect__tokenizer'   : [tokenizer, tokenizer_porter],
        'clf__kernel' : ['linear'],
        'clf__degree' : [1, 2, 3, 4, 5],
        'clf__C'      : scipy.stats.loguniform(0.0001, 1000.0),
    },
    {
        'vect__ngram_range' : [(1, 1), (1, 2)],
        'vect__stop_words'  : [None, new_stop],
        'vect__tokenizer'   : [tokenizer, tokenizer_porter],
        'clf__kernel' : ['rbf', 'poly', 'sigmoid'],
        'clf__gamma'  : ['scale','auto'],
        'clf__C'      : scipy.stats.loguniform(0.0001, 1000.0),
    },
    {
        'vect__ngram_range' : [(1, 1), (1, 2)],
        'vect__stop_words'  : [None, new_stop],
        'vect__tokenizer'   : [tokenizer, tokenizer_porter],
        'clf__kernel' : ['rbf', 'poly', 'sigmoid'],
        'clf__gamma'  : scipy.stats.loguniform(0.0001, 1000.0),
        'clf__C'      : scipy.stats.loguniform(0.0001, 1000.0),
    },
    {
        'vect__ngram_range' : [(1, 1), (1, 2)],
        'vect__stop_words'  : [None, new_stop],
        'vect__tokenizer'   : [tokenizer, tokenizer_porter],
        'vect__use_idf'     : [False],
        'vect__norm'        : [None],
        'clf__kernel' : ['linear'],
        'clf__degree' : [1, 2, 3, 4, 5],
        'clf__C'      : scipy.stats.loguniform(0.0001, 1000.0),
    },
    {
        'vect__ngram_range' : [(1, 1), (1, 2)],
        'vect__stop_words'  : [None, new_stop],
        'vect__tokenizer'   : [tokenizer, tokenizer_porter],
        'vect__use_idf'     : [False],
        'vect__norm'        : [None],
        'clf__kernel' : ['rbf', 'poly', 'sigmoid'],
        'clf__gamma'  : ['scale','auto'],
        'clf__C'      : scipy.stats.loguniform(0.0001, 1000.0),
    },
    {
        'vect__ngram_range' : [(1, 1), (1, 2)],
        'vect__stop_words'  : [None, new_stop],
        'vect__tokenizer'   : [tokenizer, tokenizer_porter],
        'vect__use_idf'     : [False],
        'vect__norm'        : [None],
        'clf__kernel' : ['rbf', 'poly', 'sigmoid'],
        'clf__gamma'  : scipy.stats.loguniform(0.0001, 1000.0),
        'clf__C'      : scipy.stats.loguniform(0.0001, 1000.0),
    },
]
rs_svm = RandomizedSearchCV(svm_tfidf_pipe, param_distributions,
                            scoring='accuracy', cv=5, verbose=2,
                            n_jobs = -1, random_state=42,
                            n_iter = 50, refit = True)
rs_svm.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


c:\Users\mndpp\Desktop\Github_Projects\01_movie_review_sentiment\venv\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('vect',
                                              TfidfVectorizer(lowercase=False,
                                                              min_df=5)),
                                             ('clf', SVC())]),
                   n_iter=50, n_jobs=-1,
                   param_distributions=[{'clf__C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000027C934B42B0>,
                                         'clf__degree': [1, 2, 3, 4, 5],
                                         'clf__kernel': ['linear'],
                                         'vect__ngram_range': [(1, 1), (1, 2)],
                                         'vect__stop_wo...
                                                               'nor', 'through',
                                                               'm', 'my',
                                                               'same', "you're",
                                                               'myself',
                                                               'theirs', 'do',
                                                               'the', 'herself',
                                                               'doing', 'only',
                                                               'here', 'll',
                                                               'our', 'for',
                                                               "should've",
                                                               'most', 'each',
                                                               'i', 'at',
                                                               'their', 'then', ...]],
                                         'vect__tokenizer': [<function tokenizer at 0x0000027C92EC64D0>,
                                                             <function tokenizer_porter at 0x0000027C92EC6B90>],
                                         'vect__use_idf': [False]}],
                   random_state=42, scoring='accuracy', verbose=2)

In [21]:
svm_model = post_training(rs_svm, model_name)

Best Accuracy = 0.9063
Best Parameters = 
{'clf__C': 3.4184626863138603, 'clf__degree': 1, 'clf__kernel': 'linear', 'vect__ngram_range': (1, 2), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x0000027C92EC64D0>}


c:\Users\mndpp\Desktop\Github_Projects\01_movie_review_sentiment\venv\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Accuracy on test set = 0.90804


### 4. Random Forest

In [22]:
model_name = 'random_forest'
tfidf = TfidfVectorizer(lowercase = False,
                        preprocessor=None,
                        min_df=5)
rf_tfidf_pipe = Pipeline([
    ('vect', tfidf),
    ('clf', RandomForestClassifier())
])
np.random.seed(42)
param_range = scipy.stats.loguniform(0.00001, 10000.0)
param_distributions = [
    {
        'vect__ngram_range' : [(1, 1), (1, 2)],
        'vect__stop_words'  : [None, new_stop],
        'vect__tokenizer'   : [tokenizer, tokenizer_porter],
        'clf__n_estimators' : [10, 20, 30, 40, 50, 75, 100, 150, 200],
        'clf__criterion'    : ['gini', 'entropy', 'log_loss'],
        'clf__max_depth'    : [ 2, 4, 6, 8, 10, 15, 20],
        'clf__max_features' : ['sqrt', 'log2', None]
    },
    {
        'vect__ngram_range' : [(1, 1), (1, 2)],
        'vect__stop_words'  : [None, new_stop],
        'vect__tokenizer'   : [tokenizer, tokenizer_porter],
        'vect__use_idf'     : [False],
        'vect__norm'        : [None],
        'clf__n_estimators' : [10, 20, 30, 40, 50, 75, 100, 150, 200],
        'clf__criterion'    : ['gini', 'entropy', 'log_loss'],
        'clf__max_depth'    : [ 2, 4, 6, 8, 10, 15, 20],
        'clf__max_features' : ['sqrt', 'log2', None]
    },
]
rs_rf = RandomizedSearchCV(rf_tfidf_pipe, param_distributions,
                            scoring='accuracy', cv=5, verbose=2,
                            n_jobs = -1, random_state=42,
                            n_iter = 50, refit = True)
rs_rf.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


c:\Users\mndpp\Desktop\Github_Projects\01_movie_review_sentiment\venv\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('vect',
                                              TfidfVectorizer(lowercase=False,
                                                              min_df=5)),
                                             ('clf',
                                              RandomForestClassifier())]),
                   n_iter=50, n_jobs=-1,
                   param_distributions=[{'clf__criterion': ['gini', 'entropy',
                                                            'log_loss'],
                                         'clf__max_depth': [2, 4, 6, 8, 10, 15,
                                                            20],
                                         'clf__max_features': ['sqrt', 'log2',
                                                               None],
                                         'clf__n_estimators': [10, 20, 30, 40,
                                                               50, 75, 100, 150,
                                                               2...
                                                               'nor', 'through',
                                                               'm', 'my',
                                                               'same', "you're",
                                                               'myself',
                                                               'theirs', 'do',
                                                               'the', 'herself',
                                                               'doing', 'only',
                                                               'here', 'll',
                                                               'our', 'for',
                                                               "should've",
                                                               'most', 'each',
                                                               'i', 'at',
                                                               'their', 'then', ...]],
                                         'vect__tokenizer': [<function tokenizer at 0x0000027C92EC64D0>,
                                                             <function tokenizer_porter at 0x0000027C92EC6B90>],
                                         'vect__use_idf': [False]}],
                   random_state=42, scoring='accuracy', verbose=2)

In [23]:
rf_model = post_training(rs_rf, model_name)

Best Accuracy = 0.8309
Best Parameters = 
{'vect__tokenizer': <function tokenizer_porter at 0x0000027C92EC6B90>, 'vect__stop_words': None, 'vect__ngram_range': (1, 1), 'clf__n_estimators': 150, 'clf__max_features': 'sqrt', 'clf__max_depth': 10, 'clf__criterion': 'entropy'}


c:\Users\mndpp\Desktop\Github_Projects\01_movie_review_sentiment\venv\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Accuracy on test set = 0.82644


### 5. K Nearest Neighbors

In [24]:
model_name = 'knn'
tfidf = TfidfVectorizer(lowercase = False,
                        preprocessor=None,
                        min_df=5)
knn_tfidf_pipe = Pipeline([
    ('vect', tfidf),
    ('clf', KNeighborsClassifier())
])
np.random.seed(42)
param_range = scipy.stats.loguniform(0.00001, 10000.0)
param_distributions = [
    {
        'vect__ngram_range' : [(1, 1), (1, 2)],
        'vect__stop_words'  : [None, new_stop],
        'vect__tokenizer'   : [tokenizer, tokenizer_porter],
        'clf__n_neighbors' : np.arange(1, 20).tolist(),
        'clf__weights'     : ['uniform', 'distance'],
        'clf__algorithm'   : ['auto', 'ball_tree', 'kd_tree', 'brute'],
        'clf__leaf_size'   : [10, 15, 20, 25, 30, 35, 40]
    },
    {
        'vect__ngram_range' : [(1, 1), (1, 2)],
        'vect__stop_words'  : [None, new_stop],
        'vect__tokenizer'   : [tokenizer, tokenizer_porter],
        'vect__use_idf'     : [False],
        'vect__norm'        : [None],
        'clf__n_neighbors' : np.arange(1, 20).tolist(),
        'clf__weights'     : ['uniform', 'distance'],
        'clf__algorithm'   : ['auto', 'ball_tree', 'kd_tree', 'brute'],
        'clf__leaf_size'   : [10, 15, 20, 25, 30, 35, 40]
    },
]
rs_knn = RandomizedSearchCV(knn_tfidf_pipe, param_distributions,
                            scoring='accuracy', cv=5, verbose=2,
                            n_jobs = -1, random_state=42,
                            n_iter = 50, refit = True)
rs_knn.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


c:\Users\mndpp\Desktop\Github_Projects\01_movie_review_sentiment\venv\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\mndpp\Desktop\Github_Projects\01_movie_review_sentiment\venv\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', "it'", 'onc', 'onli', 'ourselv', "she'", "should'v", 'themselv', 'thi', 'veri', 'wa', 'whi', "you'r", "you'v", 'yourselv'] not in stop_words.
  warnings.warn(
c:\Users\mndpp\Desktop\Github_Projects\01_movie_review_sentiment\venv\lib\site-packages\sklearn\neighbors\_base.py:584: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('vect',
                                              TfidfVectorizer(lowercase=False,
                                                              min_df=5)),
                                             ('clf', KNeighborsClassifier())]),
                   n_iter=50, n_jobs=-1,
                   param_distributions=[{'clf__algorithm': ['auto', 'ball_tree',
                                                            'kd_tree',
                                                            'brute'],
                                         'clf__leaf_size': [10, 15, 20, 25, 30,
                                                            35, 40],
                                         'clf__n_neighbors': [1, 2, 3, 4, 5, 6,
                                                              7, 8, 9, 10, 11,
                                                              12, 13, 14, 15,
                                                              16, 17, 18, 19],
                                         'clf__wei...
                                                               'nor', 'through',
                                                               'm', 'my',
                                                               'same', "you're",
                                                               'myself',
                                                               'theirs', 'do',
                                                               'the', 'herself',
                                                               'doing', 'only',
                                                               'here', 'll',
                                                               'our', 'for',
                                                               "should've",
                                                               'most', 'each',
                                                               'i', 'at',
                                                               'their', 'then', ...]],
                                         'vect__tokenizer': [<function tokenizer at 0x0000027C92EC64D0>,
                                                             <function tokenizer_porter at 0x0000027C92EC6B90>],
                                         'vect__use_idf': [False]}],
                   random_state=42, scoring='accuracy', verbose=2)

In [25]:
knn_model = post_training(rs_knn, model_name)

c:\Users\mndpp\Desktop\Github_Projects\01_movie_review_sentiment\venv\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Best Accuracy = 0.7963
Best Parameters = 
{'vect__tokenizer': <function tokenizer_porter at 0x0000027C92EC6B90>, 'vect__stop_words': ['he', 'both', 'any', 'has', 'ma', 'itself', 'nor', 'through', 'm', 'my', 'same', "you're", 'myself', 'theirs', 'do', 'the', 'herself', 'doing', 'only', 'here', 'll', 'our', 'for', "should've", 'most', 'each', 'i', 'at', 'their', 'then', 'she', 'until', 'ourselves', 'if', "it's", 'this', 'd', 'when', 'before', 't', 'so', "that'll", 'or', 'who', 'am', 'against', 'does', 'again', 'been', 'on', 'were', 'should', 'there', 'from', 'some', "you'll", 'yours', 'once', 'but', 'now', 'those', 'than', 'off', 'being', 'these', 'and', 'an', 'you', 'which', 'few', 'will', 'further', 'how', 'more', 'in', 'himself', 'did', 'him', 'themselves', 'while', 'own', 'was', 've', "you'd", 'hers', 'what', 'having', 'them', 'had', 'such', 'up', 'about', 'its', 'during', 'over', 'where', 'o', 'ours', 'very', 'that', 'her', 'we', 'can', 'other', "she's", 'whom', 'just', 'is', 'a', '

c:\Users\mndpp\Desktop\Github_Projects\01_movie_review_sentiment\venv\lib\site-packages\sklearn\neighbors\_base.py:584: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")


Accuracy on test set = 0.79752


### Test Accuracy for all models

In [28]:
accuracy_models_list = []
for model in accuracy_models:
    accuracy_models_list.append([model, accuracy_models[model]])
accuracy_df = pd.DataFrame(accuracy_models_list, columns=['Model Name', 'Accuracy']).sort_values(by='Accuracy', ascending=False, ignore_index=True)
accuracy_df

,Model Name,Accuracy
0,SVM,0.90804
1,logistic_regression,0.90188
2,naive_bayes,0.88232
3,random_forest,0.82644
4,knn,0.79752
